In [67]:
import csv
import json
import operator
from collections import OrderedDict, defaultdict

import pandas as pd
import re
from nltk.corpus import stopwords
from pandas.io.json import json_normalize
import pandas as pd
import re
import spacy


# def score(query_doc, gen_docs):
#     dictionary = gensim.corpora.Dictionary(gen_docs)
# 
#     corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
#     tf_idf = gensim.models.TfidfModel(corpus)
# 
#     sims = gensim.similarities.Similarity('/home/az/Documents/work/document-similarity/data/', tf_idf[corpus],
#                                           num_features=len(dictionary))
#     query_doc_bow = dictionary.doc2bow(query_doc)
#     query_doc_tf_idf = tf_idf[query_doc_bow]
#     return sims[query_doc_tf_idf].tolist()


def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)


def clean(data):
    return data.apply(lambda x: striphtml(str(x))).apply(lambda x: x.lower())


def get_data(d):
    data = df[df['innitie_category'] == d]
    values = data['short_description'].values.tolist()
    cleaned_data = [x for x in values if x != 'nan']
    return cleaned_data


def fill_matrix(matrix, list):
    for x in range(len(list)):
        arr = score(list[x], list)
        matrix[matrix.columns[x]] = arr

    last = list[-1]
    arr_add = score(last, list)
    last_index = len(list) - 1
    matrix[matrix.columns[last_index]] = arr_add
    return matrix


def create_dict(dict, items, subcat_dict):
    stop = len(dict.items())
    start = stop - items
    data = []

    for key, value in dict.items():
        for i in range(start, stop):
            vals = {'source_id': key, 'subcategory': subcat_dict[key].lower(),
                    'data': {'target_id': sorted(value.items(), key=lambda x: x[1])[i][0],
                             'score': sorted(value.items(), key=lambda x: x[1])[i][1],
                             'target_category': subcat_dict[sorted(value.items(),
                                                                   key=lambda x: x[1])[i][0]].lower()}}
            data += [vals]
    return data


stop_words = set(stopwords.words('english'))
df_innitie = pd.read_csv('/home/az/Documents/work/document-similarity/data/innitie_cataloge_20180706.csv')
level2 = df_innitie['level2'].unique().tolist()
level2_innitie = [x.lower() for x in level2 if isinstance(x, str) and len(x) >= 2]
df_unf = pd.read_excel('/home/az/Documents/work/document-similarity/data/unforgetable.xlsx')
data_unf = df_unf['Subcategory'].unique().tolist()
data_unf = [x.lower() for x in data_unf if isinstance(x, str) and len(x) >= 2]
nlp = spacy.load('en')  

tokens1 = [nlp(x) for x in level2_innitie[:5]]
tokens2 = [nlp(x) for x in data_unf[:3]]

for t in tokens1:
    for x in tokens2:
        print(t.text, x.text, t.similarity(x))


OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.